# ELT

In [21]:
# imports
import os
import sys
import pandas as pd
import seaborn as sns
import sqlite3
import warnings
from sqlite3 import Error
warnings.filterwarnings("ignore")

In [22]:
# adding project directories to system path
sys.path.insert(1, '../scripts')
import database_helpers as dh       # data base operation helper script
from constants import *

## data base connections

In [23]:
# create a connection
connection = dh.create_connection(test_data_base)

Connection to SQLite DB successful


In [24]:
# create a cursor to the previous connection
cursor = dh.get_cursor(connection)

cursor creation successful


In [25]:
# test connection cursor
dh.create_table(cursor, test_data_base_script)

table successfully created


In [26]:
data = {'product_name': ['Computer','Tablet','Monitor','Printer'],
        'price': [900,300,450,150]
        }

df = pd.DataFrame(data, columns= ['product_name','price'])
print (df)

  product_name  price
0     Computer    900
1       Tablet    300
2      Monitor    450
3      Printer    150


In [27]:
dh.df_to_sql(df, connection, "products", "append")

In [28]:
dh.load_content(cursor, "products", 2)

('Computer', 900)
('Tablet', 300)


In [29]:
data = pd.read_csv(data_set)

In [30]:
data

,track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time
0,1; Car; 48.85; 9.770344; 37.977391; 23.737688;...
1,2; Motorcycle; 98.09; 19.839417; 37.977642; 23...
2,3; Motorcycle; 63.80; 18.228752; 37.977997; 23...
3,4; Motorcycle; 145.72; 26.229014; 37.978135; 2...
4,5; Motorcycle; 138.01; 24.841425; 37.978134; 2...
...,...
917,918; Car; 78.83; 30.846243; 37.980629; 23.7350...
918,919; Motorcycle; 19.50; 9.234518; 37.979327; 2...
919,920; Car; 48.97; 24.486209; 37.978413; 23.7355...
920,921; Motorcycle; 46.68; 30.007124; 37.980020; ...


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 922 entries, 0 to 921
Data columns (total 1 columns):
 #   Column                                                                          Non-Null Count  Dtype 
---  ------                                                                          --------------  ----- 
 0   track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time  922 non-null    object
dtypes: object(1)
memory usage: 7.3+ KB


In [32]:
data.describe()

,track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time
count,922
unique,922
top,1; Car; 48.85; 9.770344; 37.977391; 23.737688;...
freq,1


### as we can see the data set is not in its right format

## compose the data set into the right format

In [33]:
columns = data.columns[0].split(";")
columns

['track_id',
 ' type',
 ' traveled_d',
 ' avg_speed',
 ' lat',
 ' lon',
 ' speed',
 ' lon_acc',
 ' lat_acc',
 ' time']

In [34]:
track_ids = []
types = []
traveled_d = []
avg_speeds = []
trajectories = []

for r in range(len(data)): 
    row = data.iloc[r,:][0].split(";")
    row_p1 = row[:4]
    row_p2 = row[4:]
    trajectory = ','.join(row_p2)
    
    track_ids.append(row_p1[0])
    types.append(row_p1[1])
    traveled_d.append(row_p1[2])
    avg_speeds.append(row_p1[3])
    trajectories.append(trajectory[1:])

In [35]:
data_dict = {columns[0]:track_ids, columns[1]:types, columns[2]:traveled_d, columns[3]:avg_speeds, columns[4]:trajectories}

In [36]:
new_df = pd.DataFrame(data_dict)
new_df.head()

,track_id,type,traveled_d,avg_speed,lat
0,1,Car,48.85,9.770344,"37.977391, 23.737688, 4.9178, 0.0518, -0.0299,..."
1,2,Motorcycle,98.09,19.839417,"37.977642, 23.737400, 16.9759, -0.0361, -0.022..."
2,3,Motorcycle,63.80,18.228752,"37.977997, 23.737264, 20.1906, -0.0795, -0.339..."
3,4,Motorcycle,145.72,26.229014,"37.978135, 23.737072, 2.7555, -0.0302, 0.0948,..."
4,5,Motorcycle,138.01,24.841425,"37.978134, 23.737103, 0.0000, 0.0000, 0.0000, ..."


In [37]:
dh.create_table(cursor, test_data_base_script)

table successfully created


In [38]:
dh.delete_table(cursor, "source")

Table dropped... 


In [39]:
dh.df_to_sql(new_df, connection, "source", "append")

In [40]:
dh.close_connection(connection)

successfully closed
